In [1]:
import tensorflow as tf
tf.__version__ 

'2.11.0'

In [2]:
# Import The Libraries 

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("C:/Users/Admin/Documents/20BLC1076/trainLabels.csv")
data.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [4]:
data['image_name'] = [i+".jpeg" for i in data['image'].values]
data.head()

,image,level,image_name
0,10_left,0,10_left.jpeg
1,10_right,0,10_right.jpeg
2,13_left,0,13_left.jpeg
3,13_right,0,13_right.jpeg
4,15_left,1,15_left.jpeg


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, val = train_test_split(data, test_size=0.2)

In [7]:
train.shape, val.shape

((28100, 3), (7026, 3))

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
import cv2
def load_ben_color(image):
    IMG_SIZE = 224
    sigmaX=10
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

In [10]:
data_gen = ImageDataGenerator(rescale=1/255.,
                              zoom_range=0.15,
                              fill_mode='constant',
                              cval=0.,
                              horizontal_flip=True,
                              vertical_flip=True,
                              preprocessing_function=load_ben_color)

In [11]:
# batch size
bs = 128

train_gen = data_gen.flow_from_dataframe(train, 
                                         "C:/Users/Admin/Documents/20BLC1076/resized_train/resized_train",
                                         x_col="image_name", y_col="level", class_mode="raw",
                                         batch_size=bs,
                                         target_size=(224, 224))
val_gen = data_gen.flow_from_dataframe(val,
                                       "C:/Users/Admin/Documents/20BLC1076/resized_train/resized_train",
                                       x_col="image_name", y_col="level", class_mode="raw",
                                       batch_size=bs,
                                       target_size=(224, 224))

Found 28100 validated image filenames.
Found 7026 validated image filenames.


In [12]:
resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
    )

In [13]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [14]:
import keras.layers as L
from keras.models import Model
from keras.callbacks import EarlyStopping

In [15]:
for layer in resnet.layers:
    layer.trainable = False

In [16]:
x = resnet.output
y = resnet.output

#Pooling Layer
x = L.GlobalMaxPooling2D()(x)
y = L.GlobalAveragePooling2D()(y)

#Flattening Layer
x = L.Flatten()(x)
y = L.Flatten()(y)

#Batch Normalization
x = L.BatchNormalization()(x)
y = L.BatchNormalization()(y)

#Concatenation
output = tf.keras.layers.concatenate(
    [x, y], axis=1       #concatenate along row axis.
)


output = L.Dropout(0.50)(output)
output = L.Dense(1024, activation="relu")(output)
output = L.Dropout(0.50)(output)
output = L.Dense(64, activation="relu")(output)
predictions = L.Dense(5, activation='softmax')(output)





In [17]:
print(predictions)

KerasTensor(type_spec=TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), name='dense_2/Softmax:0', description="created by layer 'dense_2'")


In [18]:
tf.data.experimental.enable_debug_mode()

In [19]:
model = Model(inputs=resnet.input, outputs=predictions)

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [22]:
history = model.fit(
    train_gen,
    validation_data = val_gen,
    epochs = 1,
    steps_per_epoch = 220,
    validation_steps = 55,
    callbacks = [callback],
    verbose = 1
)






220/220 [==============================] - 1495s 7s/step - loss: 0.9671 - accuracy: 0.7248 - val_loss: 0.9374 - val_accuracy: 0.7326


In [23]:
output_tensors = model.output

# Create a Keras function to extract the tensors
extract_output = tf.keras.backend.function([model.input], [output_tensors])


In [25]:
# Generate a sample input for testing
sample_input = np.random.randn(35126, 224, 224, 3)  # Adjust the shape according to your input

In [ ]:
# Get the output tensors for the sample input
output_arrays = extract_output([sample_input])[0]

In [ ]:
# Print the arrays
print(output_arrays)

In [ ]:
len(output_arrays)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
# Assuming `feature_vectors` represents your extracted feature vectors
feature_vectors = output_arrays

In [ ]:
# Assuming `labels` represents the corresponding labels for the feature vectors
labels = data.level

In [ ]:
# Preprocess the feature vectors if needed (e.g., scaling or normalization)

In [ ]:
# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)


In [ ]:
# Train the SVM classifier
svm_classifier = SVC()
svm_classifier.fit(x_train, y_train)


In [ ]:
# Make predictions on the test set
y_pred = svm_classifier.predict(x_test)


In [ ]:
# Evaluate the performance of the SVM classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
